In [9]:
import pandas as pd
import pickle

# Sample input data (replace with realistic values)
sample_data = pd.DataFrame({
    'Temperature': [28.0],  # Example temperature
    'Humidity': [25.0],     # Example humidity
    'Wind': [5],            # Example wind direction (mapped value)
    'Wind Speed': [7.0],   # Example wind speed
    'Condition': [1],       # Example weather condition (mapped value)
    'pH (units)': [8.5],    # Example pH level
    'Ammonia (mg/L)': [0.031], # Example ammonia level
    'Nitrate (mg/L)': [0.05], # Example nitrate level
    'Inorganic Phosphate (mg/L)': [0.27], # Example phosphate level
    'BOD (mg/l)': [2.0],    # Example BOD level
    'Dissolved Oxygen (mg/l)': [8.5], # Example dissolved oxygen level
    'Total coliforms (MPN/100ml)': [2.5] # Example total coliforms
})

# Load the scaler
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

# Standardize the sample input using the same scaler
sample_data_scaled = scaler.transform(sample_data)

# Load the saved model
with open('best_svr_model.pkl', 'rb') as file:
    loaded_svr_model = pickle.load(file)

# Predict using the loaded model
sample_prediction = loaded_svr_model.predict(sample_data_scaled)

# Print the prediction
print(f'Sample Prediction: {sample_prediction[0]}')


Sample Prediction: 14728.006966535662
